In [16]:
from dotenv import load_dotenv
from langchain.agents import create_react_agent
from langchain_community.chat_models import ChatOllama
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_openai import AzureChatOpenAI
import os
load_dotenv()

True

In [25]:


llm = AzureChatOpenAI(
    deployment_name="gpt-4",
    model_name="gpt-4", 
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version=os.getenv('OPENAI_API_VERSION'),
    openai_api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    temperature=0
)

server_params = StdioServerParameters(
    command="python",
    args=["server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # 🔍 DEBUG: Check what tools are available from MCP server
        tools_response = await session.list_tools()  # This returns a response object
        available_tools = tools_response.tools         # Access the .tools property
        
        print(f"📋 Available MCP tools: {[tool.name for tool in available_tools]}")
        print(f"📊 Number of tools: {len(available_tools)}")
        
        # Check tool details
        for tool in available_tools:
            print(f"🛠️ Tool: {tool.name} - {tool.description}")
        
        # Load tools for LangChain
        tools = await load_mcp_tools(session)
        print(f"🔧 Loaded LangChain tools: {[tool.name for tool in tools]}")
        print(f"📊 Number of LangChain tools: {len(tools)}")
        
        agent = create_react_agent(model=llm, tools=tools)
        
        # Try a more explicit search query
        agent_response = await agent.ainvoke({
            "messages": [{"role": "user", "content": "Search for information about 'model context protocol' using web search"}]
        })
        
        print("\n" + "="*50)
        print("AGENT RESPONSE:")
        print("="*50)
        print(agent_response)

📋 Available MCP tools: ['web_search', 'news_search', 'smart_search', 'research_search']
📊 Number of tools: 4
🛠️ Tool: web_search - 
    Perform a web search using Brave Search API
    
    Args:
        query: Search query string
        count: Number of results to return (1-20)
        offset: Number of results to skip for pagination
        country: Country code for localized results (e.g., "US", "GB", "CA")
        search_lang: Language for search results (e.g., "en", "es", "fr")
        freshness: Time filter ("pd" for past day, "pw" for past week, "pm" for past month, "py" for past year)
    
    Returns:
        List of search results with title, URL, description, and metadata
    
🛠️ Tool: news_search - 
    Search for recent news articles using Brave Search API
    
    Args:
        query: Search query string
        count: Number of results to return (1-20)
        offset: Number of results to skip for pagination
        country: Country code for localized results
        sea